In [2]:
import torch
import snntorch

In [52]:

output_spikes = torch.tensor([[
    [0.0, 0.0, 1.0, 0.0, 0.0],
    [1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 1.0],
    [0.0, 0.0, 0.0, 0.0, 1.0],
    [1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
]])

print(output_spikes.shape)

torch.Size([1, 8, 5])


In [22]:
def spike_counting_decoding(spikes):
    spike_counts = torch.sum(spikes, dim=1)
    action = torch.zeros(spikes.size(0))
    max_spike_count = torch.max(spike_counts)
    candidates = torch.where(spike_counts == max_spike_count)[0]
    if len(candidates) > 1:
        action[torch.multinomial(candidates.float(), 1)] = 1
    else:
        action[candidates] = 1
    return action

In [45]:
def decode_time_to_first_spike(spike_trains):
    
    decoded_values = torch.full((spike_trains.size(0),), spike_trains.shape[0] + 1)
    print(decoded_values)

    for i, train in enumerate(spike_trains):

        first_spike_time = torch.argmax(train) 

        if train[first_spike_time] == 1:
            decoded_values[i] = first_spike_time.float() + 1

    return decoded_values

In [74]:
def decode_first_spike(spike_trains):
    decoded_vector = [spike_trains.size(0)+1] * spike_trains.size(1)
    
    for neuron_idx in range(spike_trains.size(1)):
        first_spike = (spike_trains[:, neuron_idx] == 1).nonzero(as_tuple=True)[0]
        if first_spike.nelement() != 0:
            decoded_vector[neuron_idx] = first_spike[0].item() + 1
    
    return torch.FloatTensor(decoded_vector)

In [75]:
decoded_trains = decode_first_spike(output_spikes)

print(decoded_trains)

tensor([1., 1., 1., 2., 1.])


In [41]:
# Decode the output spikes
decoded_output = decode_time_to_first_spike(output_spikes)
print(decoded_output)

tensor([9, 9, 9, 9, 9, 9, 9, 9])
tensor([3, 1, 2, 9, 5, 5, 1, 9])


In [43]:

p_actions = torch.nn.functional.softmax(-decoded_trains, dim=0)
print(p_actions)

tensor([2.4173e-01, 8.8927e-02, 6.5709e-01, 2.2043e-04, 1.2035e-02])
